In [30]:

USE_WIDGETS = True

import csv as csv


if USE_WIDGETS:
    %matplotlib widget
else:
    %matplotlib inline
    
import matplotlib as mpl

import matplotlib.pyplot as plt




import numpy as np
import matplotlib.pyplot as plt


import scipy

plt.rcParams.update({
  "text.usetex": True,
  "font.family": "Helvetica",
  "figure.max_open_warning" : 1000,
  "font.size": 10
})

def shift(xs, n, num):
    
    e = np.empty_like(xs)
    if(xs.ndim == 2):
        a, b = xs.shape
        if(b > a):
            if n >= 0:
                e[0][:n] = num
                e[0][n:] = xs[0][:-n]
            else:
                e[0][n:] = num
                e[0][:n] = xs[0][-n:]
        else:
            if n >= 0:
                e[:n][0] = num
                e[n:][0] = xs[:-n][0]
            else:
                e[n:][0] = num
                e[:n][0] = xs[-n:][0]
            
    
    else:
        if n >= 0:
            e[:n] = num
            e[n:] = xs[:-n]
        else:
            e[n:] = num
            e[:n] = xs[-n:]
    return e

def ACF(x):
    

    if(x.ndim == 1):
        x = np.atleast_2d(x)
    
    b, N = x.shape
    if(b > N):
        N = b
        x = np.transpose(x)
    
    xt = np.conjugate(x)

    xt_c = np.transpose(x)

    
    r = np.zeros((2*N - 1, 1), dtype = complex)
    
    
    for k in range(N):
        r[N-1 + k] = xt @ xt_c / N
        xt[0][N-(1 + k)] = 0 #x = [x*[0] ... x*[N-1 -k] 0 ... 0]
        xt_c = np.roll(xt_c, -1) #x = [x[k] x[k+1] ... x[N-1] x[0] ... x[k-1]]
    
    for k in range(N):
        r[k] = np.conjugate(r[-(1 + k)])
        
    return np.concatenate(r)
     

    

    


In [82]:
import csv as csv
file = open('P01_test_sekvenca_1.csv')
filename = 'P01_test_sekvenca_1.csv'

csvreader = csv.reader(file)

x = np.array([])
for row in csvreader:
        x = np.append(x, complex(row[0]))


file.close()


In [87]:
r = ACF(x)
N = (len(r) - 1)//2

r_r = np.real(r)
r_i = np.imag(r)
n = np.arange(len(r)) - (len(r) - 1)//2
k = 10
n = np.arange(-k, k + 1)

(fig1, (ax1, ax2)) = plt.subplots(2,1)
ax1.set_title('Pomerena procena ACF')
ax1.set_ylabel('$Re\hat{r}_{xx}[k]$')
ax1.set_xlabel('$n$')



ax1.plot(n, r_r[N-1 - k + 1 : N-1 + k + 2])


ax2.set_ylabel('$Im\hat{r}_{xx}[k]$')
ax2.set_xlabel('$n$')
ax2.plot(n, r_i[N-1 - k + 1: N-1 + k + 2])

fig1.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
def FT_ACF(x, f):
    N = (len(x)-1)//2
    n = np.atleast_2d(np.arange(-N, N+1))
    xt = np.atleast_2d(x)
    ft = np.atleast_2d(f)
    
   
    
    #we neeed x in form n x 1
    a, b= xt.shape
    if(b > a):
        xt = np.transpose(xt)
        
    #We need ft in form m x 1
    a, b = ft.shape
    if(b > a):
        ft = np.transpose(ft)

    Wn = np.exp(-2j*np.pi*(ft @ n))
    
    X = Wn @ xt

    return X

def perBT(x, w, f):

    
    r = ACF(x)

    N = len(r)
    M = len(w)
    #mora da vazi M mod 2 == 1!
    k = (N-M)//2
 
    w = np.pad(w, (k, k), 'constant')
  
    #print(N, M)
    #print(w)
    #print(r)
  

    X = FT_ACF(r*w, f)
    
    Pper = np.real(X)
    
    return np.concatenate(Pper)

def FT(x, f):
    N = len(x)
    n = np.atleast_2d(np.arange(N))
    xt = np.atleast_2d(x)
    ft = np.atleast_2d(f)
    
   
    
    #we neeed x in form n x 1
    a, b= xt.shape
    if(b > a):
        xt = np.transpose(xt)
        
    #We need ft in form m x 1
    a, b = ft.shape
    if(b > a):
        ft = np.transpose(ft)

    Wn = np.exp(-2j*np.pi*(ft @ n))
    
    X = Wn @ xt

    return X

def per(x, f):
    if(x.ndim == 2):
        x = np.concatenate(x)
    
    N = len(x)

    X = FT(x, f)
    
    Pper = np.power(np.abs(X), 2)/N
    
    return np.concatenate(Pper)

In [94]:
f = np.atleast_2d(np.linspace(-0.5, 0.5, num = 50))
M = 21
w = np.bartlett(M)



Pbt = perBT(x, w, f)
#print(x)
(fig1, ax1) = plt.subplots(1,1)
ax1.set_title('Periodogram BT')
ax1.set_ylabel('$\hat{P}_{bt}(f)$')
ax1.set_xlabel('$f$')

f = np.concatenate(f)
ax1.plot(f, Pbt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
Nr = 1000
N = 16
f = 0.15
K = 2
Pbt = []
M = 11
w = np.bartlett(M)
n = np.arange(N)
Per = []

for i in range(Nr):
    e = np.random.standard_normal(size = (1, N))
    fi = np.random.uniform(low = 0.0, high = 2*np.pi)
    x = np.sin(2*np.pi*n*0.2 + fi) + e
    Pbt.append(perBT(x, w, f))
    Per.append(per(x, f))




In [106]:
import seaborn as sns

estimations = np.concatenate(Pbt)




estimations.sort()

q75, q25 = np.percentile(estimations, [75 ,25])
IQR = q75 - q25

SIZE = len(estimations)
MAX = estimations[SIZE-1]
MIN = estimations[0]

k = 2*IQR*(SIZE**(-1/3))


(fig1, ax1) = plt.subplots(1,1)

sns.histplot(estimations, bins = int((MAX - MIN)/k))

ax1.set_title('Histogram estimacija $\hat{P}_{bt}(f)$')
ax1.set_ylabel(' ')
ax1.set_xlabel('$n$')



mean = np.mean(estimations)
var = np.var(estimations, ddof = 1)


textstr = '\n'.join((
    r'$\mu=%.2f$' % (mean, ),
    r'$\sigma=%.2f$' % (np.sqrt(var), )))

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax1.text(0.5, 0.95, textstr, transform=ax1.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

estimations = np.concatenate(Per)


estimations.sort()

q75, q25 = np.percentile(estimations, [75 ,25])
IQR = q75 - q25

SIZE = len(estimations)
MAX = estimations[SIZE-1]
MIN = estimations[0]

k = 2*IQR*(SIZE**(-1/3))


(fig1, ax1) = plt.subplots(1,1)

sns.histplot(estimations, bins = int((MAX - MIN)/k))

ax1.set_title('Histogram estimacija $\hat{P}_{bt}(f)$')
ax1.set_ylabel(' ')
ax1.set_xlabel('$n$')



mean = np.mean(estimations)
var = np.var(estimations, ddof = 1)


textstr = '\n'.join((
    r'$\mu=%.2f$' % (mean, ),
    r'$\sigma=%.2f$' % (np.sqrt(var), )))

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax1.text(0.5, 0.95, textstr, transform=ax1.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.95, '$\\mu=1.21$\n$\\sigma=1.22$')

In [112]:
import scipy.signal as sig


Nr = 50
N = 49
f = np.linspace(0, 0.5, 50)
K = [21, 49, 95]
n = np.arange(N)
Pbt= []

Pall = []
for j in range(len(K)):
    w = sig.parzen(K[j])
    Pbt = []
    for i in range(Nr):
        e = np.random.standard_normal(size = (1, N))
        fi1 = np.random.uniform(low = 0.0, high = 2*np.pi)
        fi2 = np.random.uniform(low = 0.0, high = 2*np.pi)
        x = np.sin(2*np.pi*n*0.2 + fi1) + np.sin(2*np.pi*n*0.3 + fi2) + e
        Pbt.append(perBT(x, w,f))
    Pall.append(Pbt)
    
(fig1, (ax1, ax2)) = plt.subplots(2, 1)
ax1.set_title('Primena zatvaranja prozora, sr vr')
ax1.set_ylabel(' ')
ax1.set_xlabel('$f$')

ax2.set_title('S.T.D')
ax2.set_ylabel(' ')
ax2.set_xlabel('$f$')



for i in range(len(K)):
    
    
    suma = 0
    Pmean = []
    var = 0 
    Pvar = []
    for j in range(len(f)):
        for k in range(Nr):
            suma = suma + Pall[i][k][j]
        suma = suma/Nr
        for k in range(Nr):
            var = var +  (Pall[i][k][j] - suma)**2
        var = var/(Nr-1)
        var = np.sqrt(var)
        Pmean.append(suma)
        Pvar.append(var)
        
    ax1.plot(f, Pmean, label = f"M = {(K[i] - 1)//2}")
    ax2.plot(f, Pvar, label = f"M = {(K[i] - 1)//2}")
    

fig1.tight_layout()
plt.legend(loc="upper right")
ax1.legend(loc = "upper right")
    
     
    
        
    
    



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
from scipy.stats import chi2
Nr = 200
K = 2
f = 0.25
Pavg = []
M = 8
N = 2*M
w = np.bartlett(2*M+1)

for j in range(Nr):
    x = np.zeros((N, 1))
    x[0] = np.random.standard_normal()
    
    for i in range(N-1):
        x[i + 1] = 0.6*x[i] + np.random.standard_normal()
        
    Pavg.append(perBT(x, w, f))
    

    
Preal = 1/(1.36 - 1.2*np.cos(2*np.pi*f ))
errordB = []

v = (2*N)/sum(np.power(w, 2))
for i in range (Nr):
    errordB.append(10*np.log10(Preal) - 10*np.log10(Pavg[i][0]))

aplha = 0.05

import scipy.stats as stats
                
upper = 10*np.log10(v/stats.chi2.ppf(df=v, q=aplha/2))
lower =10*np.log10(stats.chi2.ppf(df=v, q=1 - aplha/2)/v)
    
    

In [120]:
import seaborn as sns

estimations = errordB




estimations.sort()

q75, q25 = np.percentile(estimations, [75 ,25])
IQR = q75 - q25

SIZE = len(estimations)
MAX = estimations[SIZE-1]
MIN = estimations[0]

k = 2*IQR*(SIZE**(-1/3))


(fig1, ax1) = plt.subplots(1,1)

sns.histplot(estimations, bins = int((MAX - MIN)/k))

ax1.set_title('Histogram estimacija $\hat{P}_{PER}(f)$')
ax1.set_ylabel(' ')
ax1.set_xlabel('$n$')



mean = np.mean(estimations)
var = np.var(estimations, ddof = 1)


textstr = '\n'.join((
    r'$\mu=%.2f$' % (mean, ),
    r'$\sigma=%.2f$' % (np.sqrt(var), )))

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax1.text(0.5, 0.95, textstr, transform=ax1.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

plt.plot([mean +upper, mean + upper], [0, 35])
plt.plot([mean - lower, mean - lower], [0, 35])





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:

f = np.linspace(0, 0.5, 50)
M = 30
w = np.bartlett(2*M+1)
N = 1000
x = np.zeros((N, 1))

x[0] = np.random.standard_normal()

    
for i in range(N-1):
    x[i + 1] = 0.6*x[i] + np.random.standard_normal()
    
v = (2*N)/sum(np.power(w, 2))
        
Pavg = 10*np.log10(perBT(x, w, f))
Preal = 10*np.log10(1/(1.36 - 1.2*np.cos(2*np.pi*f )))

upper = 10*np.log10(v/stats.chi2.ppf(df=v, q=aplha/2))
lower =10*np.log10(stats.chi2.ppf(df=v, q=1 - aplha/2)/v)
    
fig, ax = plt.subplots()
ax.plot(f,Pavg)
ax.plot(f, Preal)
ax.fill_between(f, (Pavg-lower), (Pavg+upper), color='b', alpha=.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …